In [1]:
import pandas as pd
import polars as pl
import numpy as np
import sys
import re
import requests 
from bs4 import  BeautifulSoup
import json
from datetime import datetime
from tqdm import trange,tnrange, tqdm_notebook
import time

In [2]:
import requests

url = "https://iboard-api.ssi.com.vn/statistics/charts/defaultAllStocksV2"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    # Process the data as needed
    print(data)
else:
    print("Failed to retrieve data. Status code:", response.status_code)

{'code': 'SUCCESS', 'message': 'Get list stocks v2 success', 'data': [{'clientName': '', 'clientNameEn': '', 'code': 'VN100', 'description': '', 'exchange': 'HOSE', 'full_name': 'VN100', 'name': 'VN100', 'securityName': '', 'stockNo': '', 'type': 'i'}, {'clientName': '', 'clientNameEn': '', 'code': 'VN30', 'description': '', 'exchange': 'HOSE', 'full_name': 'VN30', 'name': 'VN30', 'securityName': '', 'stockNo': '', 'type': 'i'}, {'clientName': '', 'clientNameEn': '', 'code': 'VNALL', 'description': '', 'exchange': 'HOSE', 'full_name': 'VNALL', 'name': 'VNALL', 'securityName': '', 'stockNo': '', 'type': 'i'}, {'clientName': '', 'clientNameEn': '', 'code': 'VNCOND', 'description': '', 'exchange': 'HOSE', 'full_name': 'VNCOND', 'name': 'VNCOND', 'securityName': '', 'stockNo': '', 'type': 'i'}, {'clientName': '', 'clientNameEn': '', 'code': 'VNCONS', 'description': '', 'exchange': 'HOSE', 'full_name': 'VNCONS', 'name': 'VNCONS', 'securityName': '', 'stockNo': '', 'type': 'i'}, {'clientName

In [7]:
#convert data từ dạng json sang dataframe
x = data['data']
df = pd.DataFrame(x)


,clientNameEn,exchange,full_name,name
0,,HOSE,VN100,VN100
1,,HOSE,VN30,VN30
2,,HOSE,VNALL,VNALL
3,,HOSE,VNCOND,VNCOND
4,,HOSE,VNCONS,VNCONS
...,...,...,...,...
1815,NaN,HOSE,CVIB2402,CVIB2402
1816,NaN,HOSE,CVNM2401,CVNM2401
1817,NaN,HOSE,CVPB2401,CVPB2401
1818,NaN,GB,GB05F2503,GB05F2503


In [33]:
url = 'https://finance.vietstock.vn/FinanceChartPage/GetListChart_Page_Mapping_ByStockCode_Full'
headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'vi',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Cookie': 'language=vi-VN; ASP.NET_SessionId=w5p2damr1ntr4z2gf4yj1nw1; __RequestVerificationToken=ek55aFae_Uak_d08MT71nEcKR8V5VHeYknAuy8EsWfe-ItLPtGpDWtFLYrOHrjwtpHjD7fpXxn90ONFQIQJkt9lzElsY__S9zOpqtdhVh1M1; __gads=ID=840b0a53678591b1:T=1719152336:RT=1719152336:S=ALNI_MbcJYzEJKJC878qaSxaWEasvrmZ2Q; __gpi=UID=00000e5fbea1a22a:T=1719152336:RT=1719152336:S=ALNI_MaL9UD21SZR5HH4MmS1cM0ccEfTTg; __eoi=ID=0d296f172e1a1026:T=1719152336:RT=1719152336:S=AA-AfjaAxdnXGrwLMZ_JmfOPSG9Y; vts_usr_lg=3FFC8DFE23415BFAE09095DA296E97BAB4A8B7200AB61B698688906B1C6C9A0CDFB290EF75C9E444B3964294A016D94F7B280B18A09232452EC80A8B44E0DFE19284D36BE5719ACCADC61948751F809647E86109E83A7B4B45FC98F1DAA3F767C3ABDA52F58268C9CB02B021F93D35ABACE8FDB7F07A0BBA9D99FABD242A210588A4DCC3E40059545E968CDCA3E47FC3; finance_viewedstock=VIC,; Theme=Light; _pbjs_userid_consent_data=3524755945110770; _ga=GA1.2.113593475.1719152336; _gid=GA1.2.921814971.1719152353; _gat_UA-1460625-2=1; dable_uid=81557631.1719152352596; cto_bundle=boVUpV8yUkdjT2l5b21aNWNGWmlaQ1VnU1JGJTJCREtnQXY0TCUyQkxKc0xsbkZJMDdtZWwlMkZadnVoTEVhZWlYcFJBZDJNS0JVSk9wRzNRQ3g5ZXpQWlZ5bXJMbFoyYWZzdXpYZlVuN1olMkJncVElMkZZZjhjRzFFaDdqQzJNTzJCTnBKMXpySFFPQkZadVM3VjNVUmNTUnBXS3hWSDB0ZkMwUTJrRHJsQ3NadFBKNFdKenM5VGVVJTNE; cto_bidid=o2YzeV9zSkI0aVc4YUlqVGN0V1dvdEl1S2tXb3BpeUx6MHF6aXpNeE1WR3JYNHJqM1UlMkJsJTJCMSUyRkowJTJCS00xdThqS0czWXZ3aU5HSGNOem5rUXVCOEhJcjZ2NzRBaFJwMXZVZlEybW1vbHNnVnElMkJLOHMlM0Q; _ga_EXMM0DKVEX=GS1.1.1719152335.1.1.1719152386.9.0.0',
    'Host': 'finance.vietstock.vn',
    'Origin': 'https://finance.vietstock.vn',
    'Referer': 'https://finance.vietstock.vn/VIC/tai-chinh.htm',
    'Sec-Ch-Ua': '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}

data = {
    'stockCode': 'VIC',
    'chartPageId': 0,
    '__RequestVerificationToken': 'KsYSYYvve574dMKlZGWKzMCvSp2NDoxFbtbXCKRc7qOkfulXh62MxMsE3wZe1PnM0-uuufVe5tgeJjx8iQMsFQPjzv-TqzHarfg4PJ1C1AkkCpSezh4f8W3pyTKGY_FiKSqb0SmJ8pQNY3lFBscyig2',
    'isCatch': True,
    'languageId': 1
}

response = requests.post(url, headers=headers, data=data)




In [37]:
response.text

'{"isSuccess":true,"data":{"InfoChart":[{"ChartMenuID":1,"ChartPageID":0,"ChartPageName":"Trang mặc định","NameChart":"Tăng trưởng doanh thu [Quý]","DispLayOrder":0,"ReportTermTypeID":2,"PeriodNumber":15,"BusinessTypeID":1,"IsEdit":0},{"ChartMenuID":2,"ChartPageID":0,"ChartPageName":"Trang mặc định","NameChart":"Tăng trưởng lợi nhuận [Quý]","DispLayOrder":1,"ReportTermTypeID":2,"PeriodNumber":15,"BusinessTypeID":1,"IsEdit":0},{"ChartMenuID":3,"ChartPageID":0,"ChartPageName":"Trang mặc định","NameChart":"Biên lợi nhuận [Quý]","DispLayOrder":2,"ReportTermTypeID":2,"PeriodNumber":15,"BusinessTypeID":1,"IsEdit":0},{"ChartMenuID":4,"ChartPageID":0,"ChartPageName":"Trang mặc định","NameChart":"Định giá [Năm]","DispLayOrder":3,"ReportTermTypeID":1,"PeriodNumber":15,"BusinessTypeID":1,"IsEdit":0},{"ChartMenuID":5,"ChartPageID":0,"ChartPageName":"Trang mặc định","NameChart":"Dòng tiền [Quý]","DispLayOrder":4,"ReportTermTypeID":2,"PeriodNumber":15,"BusinessTypeID":1,"IsEdit":0},{"ChartMenuID":6,

In [38]:
x = response.json()['data']['InfoChartDetail']

In [40]:
y = pd.DataFrame(x)

In [45]:
y

,ChartMenuID,ReportNormID,ReportNormName,NormTerm,Color,ChartType,Value,Unit,ValueSumPeriod,PeriodNumber
0,1,30,Tăng trưởng doanh thu thuần,Q3/2020,#ff6361,spline,13.76,%,13.760000,15
1,1,30,Tăng trưởng doanh thu thuần,Q4/2020,#ff6361,spline,-6.17,%,-6.170000,15
2,1,30,Tăng trưởng doanh thu thuần,Q1/2021,#ff6361,spline,51.57,%,51.570000,15
3,1,30,Tăng trưởng doanh thu thuần,Q2/2021,#ff6361,spline,65.69,%,65.690000,15
4,1,30,Tăng trưởng doanh thu thuần,Q3/2021,#ff6361,spline,-16.16,%,-16.160000,15
...,...,...,...,...,...,...,...,...,...,...
1000,1383,2210,"Phần lãi/lỗ từ công ty liên doanh, liên kết",Q1/2023,#00b7c7,columnstacked,NaN,Tỷ,4264.286915,15
1001,1383,2210,"Phần lãi/lỗ từ công ty liên doanh, liên kết",Q2/2023,#00b7c7,columnstacked,NaN,Tỷ,3672.016000,15
1002,1383,2210,"Phần lãi/lỗ từ công ty liên doanh, liên kết",Q3/2023,#00b7c7,columnstacked,NaN,Tỷ,4475.332000,15
1003,1383,2210,"Phần lãi/lỗ từ công ty liên doanh, liên kết",Q4/2023,#00b7c7,columnstacked,NaN,Tỷ,1305.848000,15


In [46]:
y[y['ChartMenuID']==1]

,ChartMenuID,ReportNormID,ReportNormName,NormTerm,Color,ChartType,Value,Unit,ValueSumPeriod,PeriodNumber
0,1,30,Tăng trưởng doanh thu thuần,Q3/2020,#ff6361,spline,13.760000,%,13.760000,15
1,1,30,Tăng trưởng doanh thu thuần,Q4/2020,#ff6361,spline,-6.170000,%,-6.170000,15
2,1,30,Tăng trưởng doanh thu thuần,Q1/2021,#ff6361,spline,51.570000,%,51.570000,15
3,1,30,Tăng trưởng doanh thu thuần,Q2/2021,#ff6361,spline,65.690000,%,65.690000,15
4,1,30,Tăng trưởng doanh thu thuần,Q3/2021,#ff6361,spline,-16.160000,%,-16.160000,15
5,1,30,Tăng trưởng doanh thu thuần,Q4/2021,#ff6361,spline,-3.810000,%,-3.810000,15
6,1,30,Tăng trưởng doanh thu thuần,Q1/2022,#ff6361,spline,-21.750000,%,-21.750000,15
7,1,30,Tăng trưởng doanh thu thuần,Q2/2022,#ff6361,spline,-63.970000,%,-63.970000,15
8,1,30,Tăng trưởng doanh thu thuần,Q3/2022,#ff6361,spline,-4.550000,%,-4.550000,15
9,1,30,Tăng trưởng doanh thu thuần,Q4/2022,#ff6361,spline,19.470000,%,19.470000,15
